# Exercises day 03

## Ex01
The event for the blocking system is has the possible following event:

* Arrival of a customer 
* Customer is served
* Customer is blocked, as all $m$ service unints is occupied.

For part 1 the arrival process is modelled as a Poisson process, and the service time distribution as exponential.

Class for service units

In [ ]:
class Event:
    def __init__(self, event_type : str, time : float, ) -> None:
        if event_type.lower() not in ['arrival', 'departure']:
            raise ValueError('event_type must be either arrival or departure')
        self.event_type = event_type
        self.time = time
       

Simulation:

In [ ]:
def blocking_simulation(
    simulation_runs : int = 10,
    m : int = 10,
    N : int = 10000,
    sample_arrival : Callable[[], float] = sample_arrival_poisson_process,
    sample_service_time : Callable[[], float] = sample_service_time_exponential
    ) -> list[float]:
    """
        A function for runinng multiple simulations of a blocking system.
        
        :param simulation_runs: number of simulations to run
        :param m: number of service units
        :param N: number of customers
        :param sample_arrival: function for sampling arrival time
        :param sample_service_time: function for sampling service time
        
        :return: list of blocked fractions
    """
    # NOTE: Maybe use burn in period...?
    
    blocked_fractions = []
    for i in range(simulation_runs):
        print(f"run {i+1}")
        custmer_count = 0
        global_time = 0
        event_counter = 0
        block_count = 0

        # lists
        event_list = []
        service_units_status = [False for _ in range(m)]    # <-- Indicates whether the service units are occupied or not
        
        # First arrival
        first_arrival = sample_arrival()
        event_list.append(Event('arrival', global_time + first_arrival))
        global_time += first_arrival
        event_list.append(Event('departure', global_time + sample_service_time()))
        service_units_status[0] = True # <-- unit 1 is occupied

        while custmer_count < N:
            
            current_event = event_list[event_counter]

            # Increment global time
            global_time = current_event.time

            if current_event.event_type == 'arrival':
                custmer_count += 1

                # Check for free service units
                indx, available = check_available_service(service_units_status)
                
                if available:
                    # Insert departure event and depend to eventlist
                    
            
                    departure_event = Event('departure', global_time + sample_service_time())
                    event_list = apend_event(event_list, departure_event)

                    # Take service unit
                    service_units_status[indx] = True # <-- unit indx is occupied

                if not available:
                    # Costumer blocked
                    block_count += 1
                
                # insert time for next arrival
                arrival_event = Event('arrival', global_time + sample_arrival())
                event_list = apend_event(event_list, arrival_event)

            elif current_event.event_type == 'departure': 
                # Free the service unit for the current departure event
                for indx, unit_occupied in enumerate(service_units_status):
                    if unit_occupied:
                        service_units_status[indx] = False # <-- unit indx is free
                        break
                        
            # increment event counter
            event_counter += 1
        
        blocked_fractions.append(block_count / N)
    
    return blocked_fractions

Exact solution

The simulation is sufficiently close to the found exact solution.

## EX02
The model is now redefined for more other arrival processes of the costumers.

### a) Erlang distributed arrival


In [ ]:
block_fraction = blocking_simulation(sample_arrival=sample_arrival_erlang)

block_fraction_conf = compute_confidence_interval_sample(block_fraction)

print(f"The fraction of blocked customers \n{block_fraction}")
print(f"Confidence interval for the mean blocking fraction: {block_fraction_conf}")

Simulation:

## Ex03

**We reuse Poisson from** *Ex01* **but then we use some new distributions for sampling the service time**
* **Constant service time**
* **Paretto distributed with $k = 1.05$ or $k = 2.05$**
* **Choose some other distributions of our own volition**

### a) Constant service time

### b) Pareto distributed


In [ ]:
k_list = np.linspace(1.05,2.05,2)

def sample_service_time_pareto(k) -> float:
    """
        Sample the service time from an pareto distribution.
        
        :return: service time (as a float) will be above 1
    """
    return scipy.stats.pareto.rvs(k)


Simulation with the different k values

In [172]:
k = 1.05
U = np.random.uniform(0.0, 1.0, 1000000)
beta = 8*((k - 1) / k)
samples = beta * (np.power(U, (-1 / k)) - 1)
#samples = beta * (U**(-1 / k) - 1)

print(samples.mean())

3.232770647991916
